<a href="https://colab.research.google.com/github/lucaCosca/JetNet_ML-Final-Project/blob/main/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ResNet-34 Architechture


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

In [ ]:
# Residual Block Layer definition

class ResidualBlock(tf.keras.layers.Layer):

  def __init__(self, filters, strides=1, **kwargs):
    super().__init__(**kwargs)

    self.main_layers = [
        tf.keras.layers.Conv2D(filters=filters, kernel_size=3, strides=strides, padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.Conv2D(filters=filters, kernel_size=3, strides=1, padding='same'),
        tf.keras.layers.BatchNormalization()
    ]

    self.skip_layers = []
    if strides > 1:
      self.skip_layers = [
          tf.keras.layers.Conv2D(filters=filters, kernel_size=1, strides=strides, padding='same'),
          tf.keras.layers.BatchNormalization()
      ]

  def call(self, inputs):
    Z = inputs
    for layer in self.main_layers:
      Z = layer(Z)

    skip_Z = inputs
    for layer in self.skip_layers:
      skip_Z = layer(skip_Z)

    return tf.keras.layers.Activation('relu')(Z + skip_Z)

In [ ]:
ResNet34 = tf.keras.Sequential([

    tf.keras.layers.Input(shape=[50, 50, 1]),
    tf.keras.layers.Conv2D(filters=64, kernel_size=7, strides=2, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(),

    ResidualBlock(filters=64),
    ResidualBlock(filters=64),
    ResidualBlock(filters=64),

    ResidualBlock(filters=128, strides=2),
    ResidualBlock(filters=128),
    ResidualBlock(filters=128),
    ResidualBlock(filters=128),

    ResidualBlock(filters=256, strides=2),
    ResidualBlock(filters=256),
    ResidualBlock(filters=256),
    ResidualBlock(filters=256),
    ResidualBlock(filters=256),
    ResidualBlock(filters=256),

    ResidualBlock(filters=512, strides=2),
    ResidualBlock(filters=512),
    ResidualBlock(filters=512),

    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(5, activation='softmax')
])